# Predicting Flight Delays

# 0. SETTINGS

## 0-1. Import packages and modules

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#font configuration
import matplotlib.font_manager as fm
font_location = "/Library/Fonts/AppleGothic.ttf"
font_name = fm.FontProperties(fname=font_location).get_name()
mpl.rc('font', family=font_name)


## 0-2. Import CSV files

In [1]:
data_flights = pd.read_csv('./flights.csv')
data_airplanes = pd.read_csv('./airlines.csv')
data_airports = pd.read_csv('./airports.csv')